# Bias-Variance分解

## 一、基本原理

假定的真实模型为$$ y = f(x) + \epsilon ,$$其中$E(\epsilon)=0,V(\epsilon)={\sigma}^{2}.$


对给定训练集$\{(x_{i},y_{i})\}$下训练出的假设$h(x)=ax+b$，我们通过预测误差$E_{P}{(y^{*}-h(x^{*}))}^{2}$对其进行评价


## 二、基本思路及待讨论解决问题

我们假定训练集样本点的概率分布满足$E_{P}[Z]=\underline{Z}$,于是
$$
\begin{split}
& E_{P}{(y^{*}-h(x^{*}))}^{2}\\
= & E[{(h(x^{*})-\underline{h(x^{*})})}^{2}] + {[\underline{h(x^{*})}-f(x^{*})]}^{2} +  E[{(y^{*}-f(x^{*}))}^{2}]\\ 
= & Variance + Bias^{2} + Noise
\end{split}
$$

然后用足够多的训练样本点$(x_i^*,y_i^*)$去评价假设$h(x)$，对每个给定的$(x_i^*,y_i^*)$，有

$$Variance_i = E[{(h(x_i^*)-\underline{h(x_i^*)})}^{2}]$$

$$Bias_i^2 = {[\underline{h(x_i^{*})}-f(x_i^{*})]}^{2}$$

于是我们可以得到对假定模型$h(x)=ax+b$最终的评价

$$Bias^2 = E[E[{(h(x_i^*)-\underline{h(x_i^*)})}^{2}]]$$

$$Variance = E[{[\underline{h(x_i^{*})}-f(x_i^{*})]}^{2}]$$


下面我们用程序来完成这个过程，并在这个基础上讨论**当误差项$\epsilon$的方差${\sigma}^{2}$变化时，$h(x)$复杂度的选择问题**。

## 三、代码部分

### 代码思路

真实函数$$ y = 2{e}^x + \epsilon ,$$其中$\epsilon\sim N(0,\sigma^2)$。对$f(x) = 2{e}^x$进行泰勒展开，有$$f(x) = 2{e}^x = 2x + 2x^2 + 2x^3 + ...$$

于是选择不同光滑度的函数h(x)进行训练，下列公式光滑度由低到高：
 $$h(x)_1 = a_1x+b$$
 $$h(x)_2 = a_1x+a_2x^2 + b$$
 $$h(x)_3 = a_1x+a_2x^2+a_3x^3+b$$

### 代码内容

#### （1）自定义函数lml
- 函数$lml$的输入为训练集TR与测试集TE，输出为以TR为训练集训练出的$h(x)$关于TE的预测集。

- TR为训练集，TE为测试集，TR和TE变量结构相同，第一个为因变量

- mlm以向前法的顺序，添加自变量，建立相应的回归模型，并给出与之对应的TE的因变量的预测值

- 预测结果排列为1个向量

In [1]:
# 建立回归函数mlm，返回 x 逐步进入的测试集的预测值y
mlm <- function(TR, TE){    # dataframe
  vname = names(TR)   # 变量名
  yn=vname[1]         # 因变量名称 y
  xn=vname[-1]        # 自变量名称 xn = c(x1,x2,x3,x4)
  mp=length(xn)       # 自变量的维数

  ypr=NULL # 预测值
  tm=paste(yn,xn[1],sep="~")     # 将因变量y和第一个自变量粘贴成y~x.1的形式，建立lm函数。
  fam=formula(tm)                # 公式化
  # 开始循环，cp为指针，从第一个自变量开始
  cp=1  
  repeat{
    lm1 = lm(fam,TR)             # 运用公式化后的lm函数对训练集TR做回归，得到相应的回归系数。
    ypr=c(ypr,predict(lm1,TE))   # 给出与之对应的TE的因变量的预测值
    
    #使用向前法，依次添加自变量x.2、x.3、……、x.p，重新建立lm函数。
    if(cp>=mp) break 
    cp=cp+1 # 加入下一个变量                  
    tm=paste(tm,xn[cp],sep="+") #  + xn
    fam=formula(tm) 
  }
  as.vector(ypr)                #返回结果为TE的因变量的预测值，其为一个拉直的向量。         
}

#### （2） 自定义函数BV
- 计算**给定光滑度和方差**后得到的**预测函数的偏差bias和方差variance**


- 有k个样本量为n的训练集dataTR，与一个样本量为N的测试集dataTE，其中p为做线性回归的自变量的最高阶数，$\sigma$为随机误差的标准差

In [1]:
library(plyr) # plyr包含apply，dpply等函数分隔操作

BV <- function(p, sigma, N, n, k){# N样本量，n训练集样本量，k训练集个数
  
  ##（1）构造两个结构相同的数据集，其中自变量维数均为p，第一个均为因变量 
    
  # 构造测试集dataTE（样本数为N）
  nx <- runif(N,-1,1) # 一个样本数为N的均匀分布的随机数
  ny <- 2*exp(nx)+rnorm(N,0,sigma)              #真实函数为y = 2*exp(x)+epsilon 其中epsilon~N(0,sigma^2)
  nz <- matrix(nx,N,p)                          #构造一个自变量矩阵，其中自变量维数为p,样本数为N
  for(i in 2:p) nz[,i] <- 10*nz[,i]*nz[,i-1]   #乘10为了量纲不要差太多，因为是(0,1)间的，平方会差很多
  dataTE <- data.frame(y=ny,z=nz)                  

  # 构造训练集dataTR（样本数为n*k）
  # (k个训练集，每个训练集的样本量均为n)
  x <- runif(n * k,-1,1) 
  y <- 2 * exp(x) + rnorm(n*k,0,sigma)
  z <- matrix(x,n*k,p)                            # ！！！！！！！！！z的长度为n*k，每一列包含k个子集
  for(i in 2:p) z[,i]=10*z[,i]*z[,i-1];
  dataTR <- data.frame(y=y,z=z)

  ## （2）
  index <- rep(1:k,rep(n,k)) # 指向属于哪个测试集1--k
 
  ## 使用自定义函数mlm，对样本量为n的训练集dataTR做关于自变量x从1到p阶的线性回归
  ## 得到与之对应的关于测试集dataTE的预测。
  ## 重复k次，计算偏差和方差。
  
  # 预测值（   k*(N*p)的矩阵   ）
  PRE=daply(dataTR, .(index), mlm, TE = dataTE)     #！！！TE为mlm的参数！！！！
                                                    # 也可以写成 index  ，等于.(index)
  # 偏差bias
  b =  ( apply(PRE,2,mean) - rep(2*exp(nx),p) )^2   # (E(h(x))-f(x))^2
  b = matrix(b, nrow=N, byrow=F) # 按列排           # N*p的矩阵，第j列的对应的回归模型的自变量最高阶数为j
  b = apply(b, 2, mean)                             # 求不同阶数对应的预测函数的偏差，E[(E(h(x))-f(x))^2]
  
  # 方差variances
  v = apply(PRE, 2, var)                            # var(h(x))
  v = matrix(v, nrow=N, byrow=F)                    # 按列排成N*p的矩阵，第j列的对应的回归模型的自变量最高阶数为j
  v = apply(v,2,mean)                               # 求不同阶数对应的预测函数的方差，E[var(h(x))]                        
  mse= b+v
  
  list(mse=mse, bias=b, var=v)
  
}

In [8]:
# 
x<-rnorm(100)
y<-x + 1 + rnorm(100,0,01)
lm(formula(y~x))
# lm(y~x) 也可以


Call:
lm(formula = formula(y ~ x))

Coefficients:
(Intercept)            x  
     0.8455       1.1702  


In [20]:
# ！！！！！！不要看
set.seed(1)
x<-matrix(rnorm(20),5)
print(x)
# apply可以用矩阵
print(apply(x,2,mean))# “2”表明对列

x[,4]<-c(0,0,0,1,1)
print(x)
# by可以用向量
print(by(x[,1],x[,4],mean))
print(as.vector(unlist(by(x[,1],x[,4],mean))))   # as.vector和unlist非常好用！！！！！

           [,1]       [,2]       [,3]        [,4]
[1,] -0.6264538 -0.8204684  1.5117812 -0.04493361
[2,]  0.1836433  0.4874291  0.3898432 -0.01619026
[3,] -0.8356286  0.7383247 -0.6212406  0.94383621
[4,]  1.5952808  0.5757814 -2.2146999  0.82122120
[5,]  0.3295078 -0.3053884  1.1249309  0.59390132
[1] 0.12926990 0.13513567 0.03812297 0.45956697
           [,1]       [,2]       [,3] [,4]
[1,] -0.6264538 -0.8204684  1.5117812    0
[2,]  0.1836433  0.4874291  0.3898432    0
[3,] -0.8356286  0.7383247 -0.6212406    0
[4,]  1.5952808  0.5757814 -2.2146999    1
[5,]  0.3295078 -0.3053884  1.1249309    1
x[, 4]: 0
[1] -0.4261464
------------------------------------------------------------ 
x[, 4]: 1
[1] 0.9623943
[1] -0.4261464  0.9623943


#### （3）模型复杂度选取


使用自定义函数，分别计算误差项方差$\sigma^2$为$1^2、5^2和10^2$时，不同光滑度对应预测函数$h(x)$的Bias和Variance，对比不同复杂度下模型的优劣。

In [1]:
#定义训练集容量n=500,k个训练集、测试集的容量N=2000、阶数p和sigma的取值
N=2000; k=2000; n=500
msigma=c(1,5,10)            
NL=length(msigma)#　sigma的长度
ps=cbind(3,msigma)# 3是变量个数

print(NL)# sigma的个数
print(ps)# 变量个数，sigma的值

[1] 3
       msigma
[1,] 3      1
[2,] 3      5
[3,] 3     10


In [33]:
# 计算不同h(x)的bias和variance
timestart <- Sys.time() #开始时间

# 每一次循环均为给定一个sigma，使用自定义函数BV计算最高阶数为p时的偏差和方差
# 一共输出三次，对应不同sigma下的mse、bias、var
for(nl in 1:NL) print(BV(ps[nl,1],ps[nl,2], N, n, k)) 

timeend <- Sys.time()   #结束时间
runningtime <- timeend-timestart
print(runningtime)      #计算运行时间

$mse
[1] 0.111930994 0.009061599 0.008314926

$bias
[1] 1.073687e-01 2.922457e-03 5.166725e-05

$var
[1] 0.004562327 0.006139142 0.008263258

$mse
[1] 0.2132227 0.1599780 0.2088457

$bias
[1] 0.1082029051 0.0029466553 0.0001472219

$var
[1] 0.1050198 0.1570314 0.2086985

$mse
[1] 0.5214544 0.6285730 0.8204290

$bias
[1] 0.1011567723 0.0029308874 0.0006573319

$var
[1] 0.4202976 0.6256421 0.8197717

Time difference of 34.07345 secs


### 结论
#### 1. 偏差Bias
Bias度量了模型预测期望函数和真实函数的偏离程度。  

在给定较低的噪声方差$sigma^2$时，**偏差会随着模型复杂度的增加而减少**；  
而当噪声方差过高时，模型复杂度的增加会导致预测函数偏离真实函数，即偏差增加。
#### 2. 方差Variance
Variance表示不同的训练数据集训练出的差异，体现了模型的**稳定性**。  
随着模型复杂度的增加，模型的简洁和稳定性均会下降，这体现在方差的增加上。
#### 3、AIC
AIC体现了模型的**拟合效果**，越小越好。